In [2]:
# %%
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
import pathlib
import json
import numpy as np
import random
import pandas as pd
# Setting the seed
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# %%
# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, kernel_size=3, num_neurons=128, conv_neurons=32, num_layers=2):
        super(SimpleCNN, self).__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Conv2d(3, conv_neurons, kernel_size=kernel_size, stride=1, padding=1))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

        # Intermediate layers
        for _ in range(num_layers - 1):
            self.layers.append(nn.Conv2d(conv_neurons, conv_neurons * 2, kernel_size=kernel_size, stride=1, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            conv_neurons *= 2

        # Calculate the size of the flattened output
        self.flattened_size = self._get_flattened_size(3, 32, kernel_size, num_layers)

        self.fc1 = nn.Linear(self.flattened_size, num_neurons)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(num_neurons, 10)

    def _get_flattened_size(self, input_channels, input_dim, kernel_size, num_layers):
        """Dynamically calculate the flattened size after conv and pooling layers."""
        x = torch.zeros((1, input_channels, input_dim, input_dim))
        for layer in self.layers:
            x = layer(x)
        return x.numel()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Deep Ensemble Model
# class DeepEnsemble:
#     def __init__(self, base_model_class, num_models, *model_args, **model_kwargs):
#         self.models = [base_model_class(*model_args, **model_kwargs) for _ in range(num_models)]


class DeepEnsemble:
    def __init__(self, base_model_class, num_models, device, *model_args, **model_kwargs):
        self.models = [base_model_class(*model_args, **model_kwargs).to(device) for _ in range(num_models)]

    def train(self, trainloader, criterion, optimizers, epochs, device):
        for epoch in range(epochs):
            for model, optimizer in zip(self.models, optimizers):
                model.train()
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            print(f"Epoch {epoch + 1}/{epochs} completed!")

    def predict(self, inputs):
        inputs = inputs.to(next(self.models[0].parameters()).device)  # Move inputs to the same device as the model
        predictions = [torch.softmax(model(inputs), dim=1) for model in self.models]
        return torch.mean(torch.stack(predictions), dim=0)
# %%

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Create validation set with 500 samples per class
def create_validation_set(trainset, samples_per_class):
    indices = []
    class_counts = {i: 0 for i in range(10)}
    for idx, (_, label) in enumerate(trainset):
        if class_counts[label] < samples_per_class:
            indices.append(idx)
            class_counts[label] += 1
        if all(count >= samples_per_class for count in class_counts.values()):
            break
    val_subset = Subset(trainset, indices)
    remaining_indices = [idx for idx in range(len(trainset)) if idx not in indices]
    train_subset = Subset(trainset, remaining_indices)
    return train_subset, val_subset

# %%

# Training and evaluation for different sample sizes
sample_sizes = [1, 5, 10, 50, 100, 500, 2000, 4000]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# %%
ensemble_size = 1
model_number = 1

for model_number in range(1, 6):
    for size in sample_sizes:

        hp_df = pd.read_csv('../src_deep_ensemble/results/DE_HP_4000/results_4000_top_5.csv')
        hp_df.reset_index(drop=True, inplace=True)
        # Get the second best hyperparameters
        row = hp_df.iloc[model_number-1]

        hyerparameters = {
            'batch_size': int(row['batch_size']),
            'num_neurons': int(row['num_neurons']),
            'conv_neurons': int(row['conv_neurons']),
            'num_layers': int(row['num_layers']),
            'lr': row['lr'],
            'epochs': int(row['epochs']),
        }

        print(f"\Loading model with {size} samples per class model {model_number}...")
        
        # Subset dataset to include only 'size' samples per class
        indices = []
        class_counts = {i: 0 for i in range(10)}
        for idx, (_, label) in enumerate(trainset):
            if class_counts[label] < size:
                indices.append(idx)
                class_counts[label] += 1
            if all(count >= size for count in class_counts.values()):
                break

        subset = Subset(trainset, indices)
        trainloader = DataLoader(subset, batch_size=64, shuffle=True)

        # Initialize model, loss, and optimizer
        base_model_class = SimpleCNN
        ensemble = DeepEnsemble(base_model_class, ensemble_size, device, kernel_size=3, num_neurons=hyerparameters['num_neurons'], conv_neurons=hyerparameters['conv_neurons'], num_layers=hyerparameters['num_layers'])
        # def __init__(self, base_model_class, num_models, device, *model_args, **model_kwargs):

        criterion = nn.CrossEntropyLoss()
        optimizers = [optim.Adam(model.parameters(), lr=hyerparameters['lr']) for model in ensemble.models]

        ensemble.models = [model.to(device) for model in ensemble.models]

        # Load the best model
        ensemble.models[0].load_state_dict(torch.load(f"../src_deep_ensemble/models_each_model/svhn_model_{size}_samples_de_1_model_{model_number}.pth"))


        # Calculate the entropy of predictions
        def calculate_entropy(probs):
            """Calculate entropy for each sample."""
            entropy = -torch.sum(probs * torch.log(probs + 1e-12), dim=1)  # Add epsilon to avoid log(0)
            return entropy

        # Evaluate the ensemble model and compute entropy
        correct = 0
        total = 0
        all_entropies = []
        all_max_probs = []
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = ensemble.predict(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Calculate entropy
                entropies = calculate_entropy(outputs)
                all_entropies.extend(entropies.cpu().numpy())  # Collect entropies

                # Calculate max probabilities
                max_probs, _ = torch.max(outputs, dim=1)
                all_max_probs.extend(max_probs.cpu().numpy())  # Collect max probabilities
            

        accuracy = 100 * correct / total


        # Calculate mean and standard deviation of entropy
        mean_entropy = np.mean(all_entropies)
        std_entropy = np.std(all_entropies)
        mean_max_prob = np.mean(all_max_probs)
        std_max_prob = np.std(all_max_probs)
        print(f"Mean Entropy: {mean_entropy:.4f}")
        print(f"Standard Deviation of Entropy: {std_entropy:.4f}")
        print(f"Mean Max Probability: {mean_max_prob:.4f}")
        print(f"Standard Deviation of Max Probability: {std_max_prob:.4f}")

        # Save the results
        results_path = f"./results/de/svhn_results_{size}_samples_de_{model_number}.json"
        pathlib.Path("./results/de/").mkdir(parents=True, exist_ok=True)
        with open(results_path, "w") as f:
            json.dump({"mean_entropy": f"{mean_entropy:.4f}", "std_entropy": f"{std_entropy:.4f}", 
                       "mean_max_prob": f"{mean_max_prob:.4f}", "std_max_prob": f"{std_max_prob:.4f}"} , f)
        print(f"Results saved at {results_path}")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
\Loading model with 1 samples per class model 1...
Mean Entropy: 0.2238
Standard Deviation of Entropy: 0.2821
Mean Max Probability: 0.9094
Standard Deviation of Max Probability: 0.1405
Results saved at ./results/de/svhn_results_1_samples_de_1.json
\Loading model with 5 samples per class model 1...
Mean Entropy: 0.5063
Standard Deviation of Entropy: 0.4023
Mean Max Probability: 0.8036
Standard Deviation of Max Probability: 0.1882
Results saved at ./results/de/svhn_results_5_samples_de_1.json
\Loading model with 10 samples per class model 1...
Mean Entropy: 0.8563
Standard Deviation of Entropy: 0.5031
Mean Max Probability: 0.6818
Standard Deviation of Max Probability: 0.2170
Results saved at ./results/de/svhn_results_10_samples_de_1.json
\Loading model with 50 samples per class model 1...
Mean Entropy: 0.3522
Standard Deviation of Entropy: 0.3931
Mean Max Probability: 0.86